In [1]:
import asyncio
import threading
import time

In [2]:
class Test:
    def __init__(self, loop):
        self._loop = loop
        self.loop = None
        self.future = None
        threading.Thread(target=self._getter).start()
        threading.Thread(target=self._setter).start()
#         loop.run_forever()
    
    def get_future(self):
        _future = self.loop.create_future()
        self.future = asyncio.wrap_future(_future)
        print("future @<%s>: %s" % (threading.current_thread(), self.future))
        return self.future
    
    async def async_get_result(self):
        current_thread = threading.current_thread()
        print("getting result ... (@<%s>)" % current_thread)
        result = await self.get_future()
#         print("result @<%s>: " % current_thread)
    
    def _getter(self):
        print("_getter @<%s>" % threading.current_thread())
        while(not self.loop):
            print("wating loop ...")
            time.sleep(1)
        print("loop comes !")
        future = asyncio.run_coroutine_threadsafe(self.async_get_result(), self.loop)
#         assert future.result(3) == 42
    
    def _setter(self):
#         self.loop = asyncio.new_event_loop()
#         asyncio.set_event_loop(loop)
        self.loop = self._loop
        while(not self.future):
            print("waiting future ...")
            time.sleep(1)
        print("future comes, let me sleep for a while ...")
        time.sleep(2)
        print("awake !")
        print("setting result @<%s> ..." % threading.current_thread())
        self.future.set_result(42)
        print("set result: %s" % self.future)

In [3]:
loop = asyncio.get_event_loop()
test = Test(loop)

_getter @<<Thread(Thread-4, started -1332757648)>>
wating loop ...
waiting future ...


RuntimeError: Event loop is running.

loop comes !
getting result ... (@<<_MainThread(MainThread, started -1219077936)>>)
future @<<_MainThread(MainThread, started -1219077936)>>: <Future pending>
future comes, let me sleep for a while ...
awake !
setting result @<<Thread(Thread-5, started -1343247504)>> ...
set result: <Future finished result=42>
